# Harnessing Weather Insights for Accurate Energy Load Forecasting

## Download Training Data

In [12]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [27]:
# API Settings

# Transparency Portal
TRANSPARENCY_API_KEY = "DEMO_KEY"
TRANSPARENCY_PERIOD_START = "2024-01-01"
TRANSPARENCY_PERIOD_END = "2024-12-31"
TRANSPARENCY_BIDDING_ZONE = "10YCZ-CEPS-----N"

# GeoSphere
GEOSPHERE_PERIOD_START = "2024-01-01"
GEOSPHERE_PERIOD_END = "2024-12-31"
GEOSPHERE_STATIONS = "1, 105"

In [15]:
# Transparency Portal
import requests

if TRANSPARENCY_API_KEY == "DEMO_KEY":
    print("Error: Missing Transparency API key.")
else:
    api_url = "https://web-api.tp.entsoe.eu/api"
    api_params = {
        "documentType": "A73",
        "processType": "A16",
        "in_Domain": TRANSPARENCY_BIDDING_ZONE,
        "periodStart": TRANSPARENCY_PERIOD_START + "0000",
        "periodEnd": TRANSPARENCY_PERIOD_END + "0000",
        "securityToken": TRANSPARENCY_API_KEY
    }

    response = requests.get(api_url, params=api_params) # Send the request => Get XML response

    if response.status_code == 200:
        print(response.text)
    else:
        print("Error: " + str(response.status_code))
        print("Response: " + response.text)

Error: 401
Response: <html><head><title>Error</title></head><body>Unauthorized. Missing or invalid security token</body></html>


In [31]:
# GeoSphere
api_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/"
api_dataset = "klima-v2-1d"
api_params = {
    # Data to get
    "parameters": "rr,tl_mittel,bewm_mittel,so_h,vv_mittel", # Rainfall, Temperature, Cloudiness, Sunshine, Wind speed
    
    "start": GEOSPHERE_PERIOD_START,
    "end": GEOSPHERE_PERIOD_END,
    "station_ids": GEOSPHERE_STATIONS,
    "output_format": "csv",
}
# Download the data to a file
response = requests.get(api_url + api_dataset, params=api_params) # Send the request => Get GeoJSON response

if response.status_code == 200:
    with open("./data/geosphere.csv", "wb") as file:
        file.write(response.content)
    print("Data saved to file")
else:
    print("Error: " + str(response.status_code))
    print("Response: " + response.text)

Data saved to file
